In [1]:
from python_calamine.pandas import pandas_monkeypatch
import pandas as pd
import numpy as np

import os

In [2]:
DATA_PATH = os.path.join('..', 'data')
RAW_PATH = os.path.join(DATA_PATH, 'raw')
INTERMEDIATE_PATH = os.path.join(DATA_PATH, 'intermediate')

In [5]:
pandas_monkeypatch()

for year in [2013, 2017, 2021]:
    for election_round in ['1ra', '2da']:
        file = f'pres-{year}-{election_round}.xlsx'
        sheets = pd.ExcelFile(os.path.join(RAW_PATH, file)).sheet_names
        wk_votacion = pd.read_excel(
            os.path.join(RAW_PATH, file),
            sheet_name=sheets[0],
            header=6,
            engine='calamine'
        )
        votacion = wk_votacion.copy()
        votacion.columns = [
            'drop', 'region', 'senatorial', 'distrito', 'comuna', 'electoral', 'local',
            'mesa', 'drop', 'nombres', 'apellido', 'drop', 'votos', 'drop',
        ]
        votacion.drop(columns=['drop'], inplace=True)
        replace_region = {
            'METROPOLITANA DE SANTIAGO': 'METROPOLITANA',
            'DE VALPARAISO': 'VALPARAISO',
            'DEL BIOBIO': 'BIOBIO',
            'DE LA ARAUCANIA': 'ARAUCANIA',
            'DEL MAULE': 'MAULE',
            "DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS": 'OHIGGINS',
            'DE LOS LAGOS': 'LOS LAGOS',
            'DE COQUIMBO': 'COQUIMBO',
            'DE ANTOFAGASTA': 'ANTOFAGASTA',
            'DE ÑUBLE': 'ÑUBLE',
            'DE LOS RIOS': 'LOS RIOS',
            'DE TARAPACA': 'TARAPACA',
            'DE ATACAMA': 'ATACAMA',
            'DE ARICA Y PARINACOTA': 'ARICA',
            'DE MAGALLANES Y DE LA ANTARTICA CHILENA': 'MAGALLANES',
            'DE AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO': 'AYSEN'
        }
        votacion.region = votacion.region.replace(replace_region).astype('category')
        votacion.senatorial = votacion.senatorial.str.split().str[-1].astype(np.int8)
        votacion.distrito = votacion.distrito.str.split().str[-1].astype(np.int8)
        replace_comuna = {
            'LLAY-LLAY': 'LLAILLAY',
            'TREGUACO': 'TREHUACO',
            'AISEN': 'AYSEN',
        }
        votacion.comuna = votacion.comuna.replace(replace_comuna).str.strip().astype('category')
        replace_electoral = {
            'AISEN': 'AYSEN',
            'PELCHUQUAAN': 'PELCHUQUIN',
            'CURRINAA': 'CURRINE',
        }
        votacion.electoral = (
            votacion.electoral
            .str.normalize('NFKD')
            .str.encode('ascii', errors='ignore')
            .str.decode('utf-8')
            .str.replace("O'HIGGINS", 'OHIGGINS', regex=False)
            .str.replace(r'\s+', ' ', regex=True)
            .str.replace(r'\((.*?)\)', r'\1', regex=True)
            .str.replace(r'-\s*([^\-]+?)\s*-', r'\1', regex=True)
            .str.replace(r'\s*-\s*', ' - ', regex=True)
            .replace(replace_electoral)
            .str.strip()
            .astype('category')
        )
        replace_local = {
            'AISEN': 'AYSEN',
            'PELCHUQUAAN': 'PELCHUQUIN',
            'CURRINAA': 'CURRINE',
            'COMPLEJO EDUCATIVO JUAN JOSE LATORRE BENAVENT': (
                'COMPLEJO EDUCATIVO JUAN JOSE LATORRE BENAVENTE'
            ),
            'COLEGIO CARLOS CONDEL DE LA HAZA': 'COLEGIO CARLOS CONDELL DE LA HAZA',
            'COLEGIO RAO GRANDE': 'COLEGIO RIO GRANDE',
            'ESCUELA GUARDIAMARINA ERNESTO RIQUELME V': (
                'ESCUELA GUARDIAMARINA ERNESTO RIQUELME'
            )
        }
        votacion.local = (
            votacion.local
            .str.upper()
            .str.replace('Á', 'A')
            .str.replace('É', 'E')
            .str.replace('Í', 'I')
            .str.replace('Ó', 'O')
            .str.replace('Ú', 'U')
            .str.normalize('NFKD')
            .str.encode('ascii', errors='ignore')
            .str.decode('utf-8')
            .str.replace("O'HIGGINS", 'OHIGGINS', regex=False)
            .str.replace('GRAL.', 'GENERAL', regex=False)
            .str.replace('VALPARAASO', 'VALPARAISO', regex=False)
            .str.replace('DARAO', 'DARIO', regex=False)
            .str.replace('GASTRONOMAA', 'GASTRONOMIA', regex=False)
            .str.replace('AGUSTAN', 'AGUSTIN', regex=False)
            .str.replace('SALESIANOS', 'SALESIANO', regex=False)
            .str.replace(' - ', ' ', regex=False)
            .str.replace(r'\b(?:No\.?|NA|Nao|NAo|NO\.)\s*(\d+)\b', r'NRO\1', regex=True)
            .str.replace(',', ' ', regex=False)
            .str.replace(r'\s+', ' ', regex=True)
            .str.replace(r'\((.*?)\)', r'\1', regex=True)
            .str.replace(r'-\s*([^\-]+?)\s*-', r'\1', regex=True)
            .str.replace(r'\s*-\s*', ' - ', regex=True)
            .replace(replace_local)
            .str.strip()
            .astype('category')
        )
        votacion.mesa = votacion.mesa.str.strip().astype('category')
        candidato = (
            votacion.nombres.str.strip()+' '+votacion.apellido.str.strip().fillna('')
        ).astype('category')
        votacion.insert(7, 'candidato', candidato)
        votacion.drop(columns=['nombres', 'apellido'], inplace=True)
        votacion.votos = (
            votacion.votos.astype(np.uint8 if votacion.votos.max() < 256 else np.uint16)
        )
        memory = votacion.memory_usage().sum()/1024**2
        votacion.to_parquet(
            os.path.join(INTERMEDIATE_PATH, f'pres-{year}-{election_round}.parquet')
        )
        print(f'{year} {election_round}: {memory:.2f} MB')
        display(votacion.head(2))

2013 1ra: 5.83 MB


,region,senatorial,distrito,comuna,electoral,local,mesa,candidato,votos
0,TARAPACA,1,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO COLEGIO SAN ANTONIO DE MATILLA,21M,FRANCO ALDO PARISI,20
1,TARAPACA,1,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO COLEGIO SAN ANTONIO DE MATILLA,21M,MARCEL CLAUDE,4


2013 2da: 2.39 MB


,region,senatorial,distrito,comuna,electoral,local,mesa,candidato,votos
0,TARAPACA,1,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO COLEGIO SAN ANTONIO DE MATILLA,21M,MICHELLE BACHELET,119
1,TARAPACA,1,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO COLEGIO SAN ANTONIO DE MATILLA,21M,EVELYN MATTHEI,44


2017 1ra: 5.51 MB


...

,region,senatorial,distrito,comuna,electoral,local,mesa,candidato,votos
0,TARAPACA,2,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,26M,SEBASTIAN PIÑERA,89
1,TARAPACA,2,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,26M,ALEJANDRO GUILLIER,68


2021 1ra: 5.40 MB


,region,senatorial,distrito,comuna,electoral,local,mesa,candidato,votos
0,TARAPACA,2,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20M,GABRIEL BORIC,22
1,TARAPACA,2,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20M,JOSE ANTONIO KAST,36


2021 2da: 2.65 MB


,region,senatorial,distrito,comuna,electoral,local,mesa,candidato,votos
0,TARAPACA,2,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20M,GABRIEL BORIC,71
1,TARAPACA,2,2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20M,JOSE ANTONIO KAST,92
